# Implementation of a comparison of Adult Credit data across all fairness functions

In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
from copy import deepcopy

from src.util import sigmoid, load_dataset, mean_difference
from src.feature_map import IdentityFeatureMap
from src.functions import cost_utility, demographic_parity
from src.plotting import plot_mean, plot_median
from src.training import train
from src.distribution import AdultCreditDistribution, ResamplingDistribution

## Uncalibrated Score: Benefit Difference: Benefit Difference

In [21]:
def calc_benefit(decisions, y, ips_weights):
    if ips_weights is not None:
        decisions *= ips_weights

    return decisions

def fairness_gradient_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]
    benefit = calc_benefit(decisions, y, ips_weights)

    log_gradient = policy.log_policy_gradient(x, s)
    benefit_grad = log_gradient * benefit
        
    return mean_difference(benefit_grad, s)

def fairness_function(**fairness_kwargs):
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]
    benefit = calc_benefit(decisions, y, ips_weights)
        
    return mean_difference(benefit, s)

bias = True
distribution = AdultCreditDistribution(bias=bias, test_percentage=0.2)
# distribution = ResamplingDistribution(bias=bias, dataset=load_dataset("../dat/adult_credit/adult_credit.npz"), test_percentage=0.2)
dim_theta = distribution.feature_dim

print(dim_theta)

def util_func(**util_params):
    util = cost_utility(cost_factor=0.5, **util_params)
    return util

training_parameters = {
    'model':{
        'utility_function': util_func,
        'fairness_function': fairness_function,
        'fairness_gradient_function': fairness_gradient_function,
        'feature_map': IdentityFeatureMap(dim_theta),
        'learn_on_entire_history': False,
        'use_sensitve_attributes': False,
        'bias': bias,
        'initial_theta': np.zeros((dim_theta))
    },
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':50,
        'epochs': 1,
        'batch_size':32,
        'learning_rate': 0.1,
        'decay_rate': 1,
        'decay_step': 10000,
        'num_batches': 32,
        'fix_seeds': True
    },
    'test': {
        'num_samples': 600
    }
}

75


In [22]:
import matplotlib.pyplot as plt
from src.training_evaluation import Statistics

training_parameters["save_path"] = "./exp-012-adult-no-fairness"
training_parameters["model"]["initial_lambda"] = 0.0

statistics, _, run_path = train(deepcopy(training_parameters), iterations=5, asynchronous=False)

plot_mean(statistics, "{}/results_mean_time.png".format(run_path))
plot_median(statistics, "{}/results_median_time.png".format(run_path))

plt.plot(statistics.results[Statistics.X_VALUES], statistics.performance(Statistics.ACCURACY, Statistics.MEDIAN))
plt.savefig("{}/acc.png".format(run_path))

## STARTED Single training run // LR = 0.1 // TS = 50 // E = 1 // BS = 32 // NB = 32 ##
## ENDED Single training run // LR = 0.1 // TS = 50 // E = 1 // BS = 32 // NB = 32 ##


In [8]:
import matplotlib.pyplot as plt
from src.training_evaluation import Statistics

training_parameters["save_path"] = "./exp-012-adult-benefit-lambda-sweep"
lambdas = np.logspace(-1, 1, base=10, endpoint=True, num=10)
lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])
training_parameters["model"]["initial_lambda"] = lambdas

statistics, model_parameters, run_path = train(training_parameters, iterations=5, asynchronous=True)

plot_mean(statistics, "{}/results_mean_lambdas.png".format(run_path))
plot_median(statistics, "{}/results_median_lambdas.png".format(run_path))

plt.plot(statistics.results[Statistics.X_VALUES], statistics.performance(Statistics.ACCURACY, Statistics.MEDIAN))
plt.savefig("{}/acc.png".format(run_path))

---------- Training with fixed lambdas ----------


In [ ]:
training_parameters["save_path"] = "./exp-012-adult-benefit-dual-gradient"
training_parameters["model"]["initial_lambda"] = 0.0
training_parameters["lagrangian_optimization"] = {
    'iterations': 20,
    'epochs': 1,
    'batch_size':256,
    'learning_rate': 3,
    'decay_rate': 1,
    'decay_step': 1000000,
    'num_decisions': 128 * 512
}

statistics, model_parameters, run_path = train(training_parameters, iterations=30, asynchronous=True)

plot_mean(statistics, "{}/results_mean_lambdas.png".format(run_path), model_parameters=model_parameters)
plot_median(statistics, "{}/results_median_lambdas.png".format(run_path), model_parameters=model_parameters)

# Covariance of decision

In [ ]:
def calc_covariance(x, s, policy, ips_weights, decisions):    
    new_s = 1 - (2*s)
    
    if ips_weights is not None:
        mu_s = np.mean(new_s * ips_weights, axis=0)
        decisions *= ips_weights
    else:
        mu_s = np.mean(new_s, axis=0)

    covariance = (new_s - mu_s) * decisions
    return covariance

def fairness_gradient_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    ips_weights = fairness_kwargs["ips_weights"]
    decisions = fairness_kwargs["decisions"]
    
    covariance = calc_covariance(x, s, policy, ips_weights,decisions)

    log_policy_gradient = policy.log_policy_gradient(x, s)
    covariance_grad = log_policy_gradient * covariance

    return np.mean(covariance_grad, axis=0)

def fairness_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    ips_weights = fairness_kwargs["ips_weights"]
    decisions = fairness_kwargs["decisions"]
    
    covariance = calc_covariance(x, s, policy, ips_weights,decisions)
    return np.mean(covariance, axis=0)

bias = True
distribution = AdultCreditDistribution(bias=bias, test_percentage=0.2)
# distribution = ResamplingDistribution(bias=bias, dataset=load_dataset("../dat/adult_credit/adult_credit.npz"), test_percentage=0.2)
dim_theta = distribution.feature_dim

print(dim_theta)

def util_func(**util_params):
    util = cost_utility(cost_factor=0.1, **util_params)
    return util

training_parameters = {
    'model':{
        'utility_function': util_func,
        'fairness_function': fairness_function,
        'fairness_gradient_function': fairness_gradient_function,
        'feature_map': IdentityFeatureMap(dim_theta),
        'learn_on_entire_history': False,
        'use_sensitve_attributes': False,
        'bias': bias,
        'initial_theta': np.zeros((dim_theta))
    },
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':50,
        'epochs': 1,
        'batch_size':256,
        'learning_rate': 0.01,
        'decay_rate': 1,
        'decay_step': 10000,
        'num_batches': 128,
        'fix_seeds': True
    },
    'test': {
        'num_samples': 8192
    }
}

In [ ]:
training_parameters["save_path"] = "./exp-012-adult-covdecision-lambda-sweep"
lambdas = np.logspace(-2, 2, base=10, endpoint=True, num=10)
lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])
training_parameters["model"]["initial_lambda"] = lambdas

statistics, model_parameters, run_path = train(training_parameters, iterations=10, asynchronous=True)

plot_mean(statistics, "{}/results_mean_lambdas.png".format(run_path))
plot_median(statistics, "{}/results_median_lambdas.png".format(run_path))

plt.plot(statistics.results[Statistics.X_VALUES], statistics.performance(Statistics.ACCURACY, Statistics.MEDIAN))
plt.savefig("{}/acc.png".format(run_path))


## Benefit difference (equality of opportunity)

In [ ]:
def calc_benefit(decisions, y, ips_weights):
    if ips_weights is not None:
        decisions *= ips_weights

    return decisions

def fairness_gradient_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]
    benefit = calc_benefit(decisions, y, ips_weights)
    
    y1_indices = np.where(y == 1)

    log_gradient = policy.log_policy_gradient(x, s)
    benefit_grad = log_gradient * benefit
        
    return mean_difference(benefit_grad[y1_indices], s[y1_indices])

def fairness_function(**fairness_kwargs):
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]
    benefit = calc_benefit(decisions, y, ips_weights)
    
    y1_indices = np.where(y == 1)
        
    return mean_difference(benefit[y1_indices], s[y1_indices])


bias = True
distribution = AdultCreditDistribution(bias=bias, test_percentage=0.2)
# distribution = ResamplingDistribution(bias=bias, dataset=load_dataset("../dat/adult_credit/adult_credit.npz"), test_percentage=0.2)
dim_theta = distribution.feature_dim

print(dim_theta)

def util_func(**util_params):
    util = cost_utility(cost_factor=0.1, **util_params)
    return util

training_parameters = {
    'model':{
        'utility_function': util_func,
        'fairness_function': fairness_function,
        'fairness_gradient_function': fairness_gradient_function,
        'feature_map': IdentityFeatureMap(dim_theta),
        'learn_on_entire_history': False,
        'use_sensitve_attributes': False,
        'bias': bias,
        'initial_theta': np.zeros((dim_theta))
    },
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':50,
        'epochs': 1,
        'batch_size':256,
        'learning_rate': 0.01,
        'decay_rate': 1,
        'decay_step': 10000,
        'num_batches': 128,
        'fix_seeds': True
    },
    'test': {
        'num_samples': 8192
    }
}


In [ ]:
training_parameters["save_path"] = "./exp-012-adult-covdecision-lambda-sweep"
lambdas = np.logspace(-2, 2, base=10, endpoint=True, num=10)
lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])
training_parameters["model"]["initial_lambda"] = lambdas

statistics, model_parameters, run_path = train(training_parameters, iterations=10, asynchronous=True)

plot_mean(statistics, "{}/results_mean_lambdas.png".format(run_path))
plot_median(statistics, "{}/results_median_lambdas.png".format(run_path))

plt.plot(statistics.results[Statistics.X_VALUES], statistics.performance(Statistics.ACCURACY, Statistics.MEDIAN))
plt.savefig("{}/acc.png".format(run_path))